In [1]:

####################################################################################################
####################################################################################################

import sys
import os.path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import display

import keras
import keras.utils.vis_utils
from keras import backend as K
from keras import layers
from keras.layers import Dense
from keras.models import load_model
from keras.models import Sequential
from keras.utils import plot_model

sys.path.append(os.path.abspath("../"))
from helpers import *

sys.path.append(os.path.abspath("../../"))
from pax_utils import s1s2_utils



Using TensorFlow backend.


In [14]:

####################################################################################################
####################################################################################################

nEventsTrain = 100000
#nEventsTrain = 1000

#file_all     = "../../data_lfs/merged_all_200k_1803261941.pkl"
file_all     = "../../pax_merge/merged/apr26/merged_all_200000.pkl"

df_all = pd.read_pickle(file_all)
df_all = df_all[df_all['intr_count'] == 1].reset_index(drop=True)
df_all   = df_all[nEventsTrain:][:]


####################################################################################################
####################################################################################################

df_train_truth = df_all[:][['x', 'y']]
df_train_input = df_all[:][s1s2_utils.getS2integralsDataFrameColumns()]


####################################################################################################
####################################################################################################

checkTrainingData(df_train_input, df_train_truth)




Input shape: (20967, 127)
Truth shape: (20967, 2)

Input DataFrame: 


,s2_area_000,s2_area_001,s2_area_002,s2_area_003,s2_area_004,s2_area_005,s2_area_006,s2_area_007,s2_area_008,s2_area_009,...,s2_area_117,s2_area_118,s2_area_119,s2_area_120,s2_area_121,s2_area_122,s2_area_123,s2_area_124,s2_area_125,s2_area_126
100000,3.759408,0.0,0.0,0.0,1.260015,3.063135,0.000000,1.025409,3.631153,2.578415,...,8.102609,0.0,18.815252,41.589948,68.044219,161.370239,300.601142,162.603203,54.638767,284.450487
100001,1.019241,0.0,0.0,0.0,0.932337,2.778582,5.607676,3.780688,0.000000,4.901986,...,5.006235,0.0,6.053813,23.788207,132.691834,44.911013,3.435027,5.428460,9.587026,23.680826
100002,0.518167,0.0,0.0,0.0,0.000000,0.000000,0.000000,3.086920,1.284619,1.386917,...,11.077027,0.0,0.294504,1.684341,1.240495,1.303689,4.023594,3.675460,4.250862,4.386411
100003,0.000000,0.0,0.0,0.0,0.000000,0.000000,1.042822,0.000000,1.662280,0.000000,...,0.000000,0.0,0.000000,1.512077,7.613226,7.268605,1.159698,3.956149,0.000000,1.534371
100004,0.000000,0.0,0.0,0.0,1.699716,0.000000,0.000000,0.000000,0.000000,0.836379,...,11.941959,0.0,0.000000,1.437122,0.915459,1.560759,0.648011,2.840589,2.020354,1.643242



Truth DataFrame:


,x,y
100000,4.270437,9.257750
100001,-29.468653,-19.037513
100002,-28.269992,3.720325
100003,16.200917,-36.713340
100004,-36.168215,-3.188817


In [16]:

####################################################################################################
####################################################################################################

model_name     = 'model_xy_s2integrals_dnnModel_elu_elu'
model_name     = "model_barge_xy_elu"
#model_name     = "model_xenon1t_elu"
model_name = 'model_xy_s2integrals_dnnModel_elu'

folder         = "models" + "/"    

model_name_h5  = folder + model_name + ".h5"
model_name_png = folder + model_name + ".png"



In [17]:

####################################################################################################
####################################################################################################

model       = load_model(model_name_h5)
arr_xy_pred = model.predict(df_train_input.values)

print("\nLoaded Model: " + model_name_h5)
print("Predicted\n")




Loaded Model: models/model_xy_s2integrals_dnnModel_elu.h5
Predicted



In [18]:

####################################################################################################
####################################################################################################

arr_x_true = df_train_truth[:]['x'].as_matrix()
arr_y_true = df_train_truth[:]['y'].as_matrix()

arr_x_pred = arr_xy_pred[:, 0]
arr_y_pred = arr_xy_pred[:, 1]

arr_dx     = arr_x_true - arr_x_pred
arr_dy     = arr_y_true - arr_y_pred



In [19]:

####################################################################################################
####################################################################################################

file_out         = "./predictions/xy_S2integrals.pkl"

cols_out         = ['event_number', 'x', 'y', 'intr_x', 'intr_y', 'intr_x_nn', 'intr_y_nn']

df_out           = df_all[:][cols_out]
df_out['x_pred'] = arr_x_pred
df_out['y_pred'] = arr_y_pred


df_out.to_pickle(file_out)


####################################################################################################
####################################################################################################

df_test = pd.read_pickle(file_out)

display(df_test[0:5][:])
print()



,event_number,x,y,intr_x,intr_y,intr_x_nn,intr_y_nn,x_pred,y_pred
100000,165454,4.270437,9.257750,1.122180,-4.364035,1.179514,-4.215021,-0.585133,-0.450990
100001,165455,-29.468653,-19.037513,10.348997,9.600877,10.638529,10.071152,-0.585133,-0.450990
100002,165458,-28.269992,3.720325,-34.288847,-2.618421,-34.155092,-2.092402,-0.585133,-0.450990
100003,165459,16.200917,-36.713340,19.825188,0.124687,20.024095,1.114278,-0.585156,-0.450985
100004,165460,-36.168215,-3.188817,-22.817669,-15.087093,-23.239706,-14.398319,-0.585160,-0.450984
